# Instalations & Imports

In [1]:
%pip install -U --quiet langchain_google_genai langchain-community langchain_chroma pypdf

In [2]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.schema import BaseChatMessageHistory, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
import os

GOOGLE_API_KEY = ""
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
# Initialize the language model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", max_tokens=1000, temperature=0)

# Construct retriever

In [4]:
dir_path = "/content/Apartments Docs"
for filename in os.listdir(dir_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(dir_path, filename)

        loader = PyPDFLoader(file_path)
        docs = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(docs)
        vectorstore = Chroma.from_documents(documents=splits, embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

retriever = vectorstore.as_retriever()

# Contextualize question

In [5]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question

In [6]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "The only topic you should assist is Apartments for sale in Israel. "
    "Remember information in chat history just if user write remember."
    "Use the following pieces of retrieved context and chat history to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Statefully manage chat history

In [7]:
class SelectiveChatMessageHistory(ChatMessageHistory):
    def add_messages(self, message: BaseMessage) -> None:
        # Check if "remember" appears anywhere in the message (case insensitive)
        if "remember" in message[0].content.lower(): # message[0].content this is the user message
            super().add_messages(message)


In [8]:
store = {}

def get_selective_session_history(session_id: str) -> SelectiveChatMessageHistory:
    if session_id not in store:
        store[session_id] = SelectiveChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_selective_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

# Test the agent

In [9]:
conversational_rag_chain.invoke(
    {"input": "My name is Yossi Gisser and i live in Lod. What is the Right to Housing and is it Implemented in Israeli Law?"},
    config={
        "configurable": {"session_id": "user123"}
    },  # constructs a key "user123" in `store`.
)["answer"]

"In Israel, the right to housing is partially protected by the Basic Law: Human Dignity and Liberty, which asserts everyone's right to dignity.  However, there's no explicit constitutional or basic law fully protecting this right, unlike some other countries with more comprehensive social rights protections.  While Israel has mechanisms like public housing and rent subsidies, enforcement can be complex.\n"

In [10]:
conversational_rag_chain.invoke(
    {"input": "What is my city?"},
    config={"configurable": {"session_id": "user123"}},
)["answer"]

'This document discusses urban renewal and gentrification in Israel in general, but it does not mention any specific city.  Therefore, I cannot tell you what your city is. More information is needed.\n'

In [11]:
import time

time.sleep(120)


In [12]:
conversational_rag_chain.invoke(
    {"input": "My name is Yossi Gisser and i live in Lod, please remember that."},
    config={
        "configurable": {"session_id": "user123"}
    },
)["answer"]

"I've remembered that your name is Yossi Gisser and you live in Lod. How can I help you with apartments for sale in Israel?\n"

In [13]:
conversational_rag_chain.invoke(
    {"input": "What is my city?"},
    config={"configurable": {"session_id": "user123"}},
)["answer"]

'Your city is Lod.\n'

As we can see, the agent remember that my (user123) city is Lod, but just if i asked him to remember.

In [14]:
import time

time.sleep(120)

In [15]:
conversational_rag_chain.invoke(
    {"input": "My name is Jon and i live in New-York, please remember that. What is the Right to Housing and is it Implemented in Israeli Law?"},
    config={
        "configurable": {"session_id": "user789"}
    },  # constructs a key "user789" in `store`.
)["answer"]

'The right to adequate housing, recognized in international human rights law, ensures affordable housing for every individual.  While Israel is party to the Universal Covenant on Economic, Social and Cultural Rights, which enshrines this right, Israeli law only partially protects it under the Basic Law: Human Dignity and Liberty.  This law primarily protects against arbitrary eviction and ensures due process.\n'

In [16]:
conversational_rag_chain.invoke(
    {"input": "What is Yossi's city and what is mine city?"},
    config={"configurable": {"session_id": "user789"}},
)["answer"]

"Your city is New York.  I have no information about Yossi's city.\n"

As we can see the agent not provide information about one user to another and not know (remember) this information.

In [17]:
import time

time.sleep(120)

In [18]:
conversational_rag_chain.invoke(
    {"input": "What is the weather in my city?"},
    config={"configurable": {"session_id": "user123"}},
)["answer"]

'I can only assist with questions related to apartments for sale in Israel.  Therefore, I cannot answer your question about the weather in Lod.\n'

We also can see that the agent doesn't answer question about another topic but still remember information that he was told to remember.